In [131]:
import pandas as pd
import numpy as np
import Levenshtein
from IPython.display import Markdown, display
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from sklearn.linear_model import LinearRegression
from sklearn.cross_validation import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import RandomForestRegressor
import matplotlib

In [132]:
def get_name(x):
    if len(x.split()) == 1:
        return x
    else:
        x = [w for w in x.split() if not any(letter in w for letter in "abcdefghijklmnopqrstuvwxyz'")]
        return " ".join(x)

In [133]:
pd.options.mode.chained_assignment = None 
Prbud = pd.read_excel(f"data/Asta Fantacalcio - Stima Prezzi di acquisto per tipo di Lega e Budget.xlsx",header=1)
F19 = pd.read_excel(f"data/F19.xlsx")
Prbud["Nome"] = Prbud["Nome"].apply(get_name)
Prbud

,RT,Squadra,Nome,Kap.,8 (%),10 (%),12 (%),Tot. (%),M.V.,Pres.
NaN,A,Juventus,CRISTIANO RONALDO,55,39.41,39.23,38.55,37.64,NaN,NaN
NaN,A,Inter,ICARDI,43,26.97,30.10,29.89,27.38,NaN,NaN
NaN,A,Lazio,IMMOBILE,43,24.75,25.33,30.39,24.72,NaN,NaN
NaN,A,Roma,DZEKO,38,19.24,20.61,21.51,20.16,NaN,NaN
NaN,A,Juventus,DYBALA,38,20.82,23.62,26.11,22.87,NaN,NaN
NaN,A,Milan,HIGUAIN,36,24.47,23.49,23.89,24.00,NaN,NaN
NaN,A,Torino,BELOTTI,33,15.83,17.67,19.43,17.16,NaN,NaN
NaN,A,Napoli,MERTENS,32,12.21,16.26,15.51,13.71,NaN,NaN
NaN,A,Napoli,INSIGNE,31,11.46,12.26,15.98,12.60,NaN,NaN
NaN,A,Torino,IAGO FALQUE,29,6.55,8.19,7.24,7.21,NaN,NaN


In [134]:
FPR19 = pd.merge(F19,Prbud,on=["Nome","Squadra"],how = 'inner')

In [135]:
list = ["R","Nome","R_Mantra","Squadra","Tit",'Quote', 'Predict', 'Predict STD',
       'FinVal', 'Diff','Tot. (%)']
FPR19 = FPR19[list]


In [136]:
FPR19["SpesaPer"] = FPR19['Tot. (%)']*3
Molt = FPR19["SpesaPer"].max()/FPR19["FinVal"].max()
FPR19["SpesaM"]=(FPR19["FinVal"]*Molt + FPR19["SpesaPer"])/2
FPR19["SpesaDiff"]=(FPR19["FinVal"]*Molt - FPR19["SpesaPer"])


In [137]:
Gruppi_Classic = FPR19.groupby("R")
Attaccanti = Gruppi_Classic.get_group("A")
Centrocampisti = Gruppi_Classic.get_group("C")
Difensori = Gruppi_Classic.get_group("D")
Portieri =Gruppi_Classic.get_group("P")

In [138]:
Attaccanti["Tier"]=Attaccanti["SpesaM"]
Attaccanti['Tier'][(Attaccanti["SpesaM"] > 55)] = "120x1"
Attaccanti['Tier'][(Attaccanti["SpesaM"] > 40) & (Attaccanti["SpesaM"]<55) & (Attaccanti["Tit"]>=0.75)] = "42x1"
Attaccanti['Tier'][(Attaccanti["SpesaM"] > 40) & (Attaccanti["SpesaM"]<55) & (Attaccanti["Tit"]<0.75)] = "24x1-NonTit"
Attaccanti['Tier'][(Attaccanti["SpesaM"] < 40) & (Attaccanti["Tit"]>=0.75)] = "24x1-Tit"
Attaccanti['Tier'][(Attaccanti["SpesaM"] <40)& (Attaccanti["Tit"]<0.75)] = "2x3"
Attaccanti["Lvl"]=Attaccanti["SpesaM"]
Attaccanti['Lvl'][(Attaccanti["SpesaM"] > 55)] = 1
Attaccanti['Lvl'][(Attaccanti["SpesaM"] > 40) & (Attaccanti["SpesaM"]<55) & (Attaccanti["Tit"]>=0.75)] = 2
Attaccanti['Lvl'][(Attaccanti["SpesaM"] > 40) & (Attaccanti["SpesaM"]<55) & (Attaccanti["Tit"]<0.75)] = 4
Attaccanti['Lvl'][(Attaccanti["SpesaM"] < 40) & (Attaccanti["Tit"]>=0.75)] = 3
Attaccanti['Lvl'][(Attaccanti["SpesaM"] <40)& (Attaccanti["Tit"]<0.75)] = 5

In [139]:
Centrocampisti["Tier"]=Centrocampisti["SpesaM"]
Centrocampisti["Tier"][Centrocampisti["SpesaM"] > 45] = "36x1"
Centrocampisti["Tier"][(Centrocampisti["SpesaM"] <45) & (Centrocampisti["SpesaM"]>30) & (Centrocampisti["Tit"]>=0.75)] = "15x1"
Centrocampisti["Tier"][(Centrocampisti["SpesaM"] <45) & (Centrocampisti["SpesaM"]>30) & (Centrocampisti["Tit"]<0.75)] = "6x2-NonTit"
Centrocampisti["Tier"][(Centrocampisti["SpesaM"] <30) & (Centrocampisti["SpesaM"]>10) & (Centrocampisti["Tit"]>=0.75)] = "6x2-Tit"
Centrocampisti["Tier"][(Centrocampisti["SpesaM"] <30) & (Centrocampisti["SpesaM"]>10) & (Centrocampisti["Tit"]<0.75)] = "5-NonTit"
Centrocampisti["Tier"][(Centrocampisti["SpesaM"] <10) & (Centrocampisti["Tit"]>=0.75)] = "5-Tit"
Centrocampisti["Tier"][(Centrocampisti["SpesaM"] <10) & (Centrocampisti["Tit"]<0.75)] = "1-NonTit"
Centrocampisti["Lvl"]=Centrocampisti["SpesaM"]
Centrocampisti["Lvl"][Centrocampisti["SpesaM"] > 45] = 1
Centrocampisti["Lvl"][(Centrocampisti["SpesaM"] <45) & (Centrocampisti["SpesaM"]>30) & (Centrocampisti["Tit"]>=0.75)] = 2
Centrocampisti["Lvl"][(Centrocampisti["SpesaM"] <45) & (Centrocampisti["SpesaM"]>30) & (Centrocampisti["Tit"]<0.75)] = 4
Centrocampisti["Lvl"][(Centrocampisti["SpesaM"] <30) & (Centrocampisti["SpesaM"]>10) & (Centrocampisti["Tit"]>=0.75)] = 3
Centrocampisti["Lvl"][(Centrocampisti["SpesaM"] <30) & (Centrocampisti["SpesaM"]>10) & (Centrocampisti["Tit"]<0.75)] = 6
Centrocampisti["Lvl"][(Centrocampisti["SpesaM"] <10) & (Centrocampisti["Tit"]>=0.75)] = 5
Centrocampisti["Lvl"][(Centrocampisti["SpesaM"] <10) & (Centrocampisti["Tit"]<0.75)] = 6

In [140]:
Difensori["Tier"]=Difensori["SpesaM"]
Difensori["Tier"][Difensori["SpesaM"] >= 35] = "9x1"
Difensori["Tier"][(Difensori["SpesaM"] <35) & (Difensori["SpesaM"]>=25) & (Difensori["Tit"]>=0.80)] = "3x4-Tit"
Difensori["Tier"][(Difensori["SpesaM"] <35) & (Difensori["SpesaM"]>=25) & (Difensori["Tit"]<0.80)] = "1x3-NonTit"
Difensori["Tier"][(Difensori["SpesaM"]<25) & (Difensori["Tit"]>=0.80)] = "1x3-Tit"
Difensori["Tier"][(Difensori["SpesaM"]<25) & (Difensori["Tit"]<0.80)] = "1x3-NonTit"
Difensori["Lvl"]=Difensori["SpesaM"]
Difensori["Lvl"][Difensori["SpesaM"] >= 35] = 1
Difensori["Lvl"][(Difensori["SpesaM"] <35) & (Difensori["SpesaM"]>=25) & (Difensori["Tit"]>=0.80)] = 2
Difensori["Lvl"][(Difensori["SpesaM"] <35) & (Difensori["SpesaM"]>=25) & (Difensori["Tit"]<0.80)] = 4
Difensori["Lvl"][(Difensori["SpesaM"]<25) & (Difensori["Tit"]>=0.80)] = 3
Difensori["Lvl"][(Difensori["SpesaM"]<25) & (Difensori["Tit"]<0.80)] = 5

In [141]:
Portieri["Tier"]=Portieri["SpesaM"]
Portieri["Tier"][Portieri["SpesaM"] >= 30] = "9x1"
Portieri["Tier"][(Portieri["SpesaM"] < 30) & (Portieri["Tit"]>=0.80)] = "2x2-Tit"
Portieri["Tier"][(Portieri["SpesaM"] < 30) & (Portieri["Tit"]<0.80)] = "2x2-NonTit"
Portieri["Lvl"]=Portieri["SpesaM"]
Portieri["Lvl"][Portieri["SpesaM"] >= 30] = 1
Portieri["Lvl"][(Portieri["SpesaM"] < 30) & (Portieri["Tit"]>=0.80)] = 2
Portieri["Lvl"][(Portieri["SpesaM"] < 30) & (Portieri["Tit"]<0.80)] = 3


In [142]:
DMant = Difensori.groupby("R_Mantra")
DMant.groups.keys()


dict_keys(['Dc', 'Dd;Dc', 'Dd;Ds;Dc', 'Dd;Ds;E', 'Dd;E', 'Dd;E;M', 'Ds;Dc', 'Ds;E', 'E'])

In [143]:
Dc = DMant.get_group("Dc").sort_values(by=["Lvl"],ascending=True)
DdDc = DMant.get_group("Dd;Dc").sort_values(by=["Lvl"],ascending=True)
DdDsDc = DMant.get_group("Dd;Ds;Dc").sort_values(by=["Lvl"],ascending=True)
DdDsE = DMant.get_group("Dd;Ds;E").sort_values(by=["Lvl"],ascending=True)
DdE = DMant.get_group("Dd;E").sort_values(by=["Lvl"],ascending=True)
DdEMdif = DMant.get_group("Dd;E;M").sort_values(by=["Lvl"],ascending=True)
DsDc = DMant.get_group("Ds;Dc").sort_values(by=["Lvl"],ascending=True)
DsE = DMant.get_group("Ds;E").sort_values(by=["Lvl"],ascending=True)
Edif = DMant.get_group("E").sort_values(by=["Lvl"],ascending=True)

In [144]:
CMant = Centrocampisti.groupby("R_Mantra")
CMant.groups.keys()

dict_keys(['C', 'C;T', 'C;W', 'C;W;T', 'Dd;E;M', 'E', 'E;C', 'E;M', 'M;C', 'T', 'T;A', 'W', 'W;A', 'W;T'])

In [145]:
C = CMant.get_group("C").sort_values(by=["Lvl"],ascending=True)
CT = CMant.get_group("C;T").sort_values(by=["Lvl"],ascending=True)
CW = CMant.get_group("C;W").sort_values(by=["Lvl"],ascending=True)
CWT = CMant.get_group("C;W;T").sort_values(by=["Lvl"],ascending=True)
DdEMcent= CMant.get_group("Dd;E;M").sort_values(by=["Lvl"],ascending=True)
Ecent= CMant.get_group("E").sort_values(by=["Lvl"],ascending=True)
EC= CMant.get_group("E;C").sort_values(by=["Lvl"],ascending=True)
EM= CMant.get_group("E;M").sort_values(by=["Lvl"],ascending=True)
MC= CMant.get_group("M;C").sort_values(by=["Lvl"],ascending=True)
T= CMant.get_group("T").sort_values(by=["Lvl"],ascending=True)
TAcent= CMant.get_group("T;A").sort_values(by=["Lvl"],ascending=True)
W= CMant.get_group("W").sort_values(by=["Lvl"],ascending=True)
WAcent= CMant.get_group("W;A").sort_values(by=["Lvl"],ascending=True)
WT= CMant.get_group("W;T").sort_values(by=["Lvl"],ascending=True)

In [146]:
AMant = Attaccanti.groupby("R_Mantra")
AMant.groups.keys()

dict_keys(['A', 'Pc', 'T;A', 'W;A'])

In [147]:
A = AMant.get_group("A").sort_values(by=["SpesaM"],ascending=False)
Pc = AMant.get_group("Pc").sort_values(by=["SpesaPer"],ascending=False)
TAatt = AMant.get_group("T;A").sort_values(by=["SpesaM"],ascending=False)
WAatt = AMant.get_group("W;A").sort_values(by=["SpesaM"],ascending=False)


In [148]:
WA = [WAatt,WAcent]
WA = pd.concat(WA).sort_values(by=["Lvl"],ascending=True)
TA = [TAatt,TAcent]
TA = pd.concat(TA).sort_values(by=["Lvl"],ascending=True)
E = [Edif,Ecent]
E = pd.concat(E).sort_values(by=["Lvl"],ascending=True)
DdEM = [DdEMdif,DdEMcent]
DdEM  = pd.concat(DdEM).sort_values(by=["Lvl"],ascending=True)


In [149]:
Aall = [WA,TA,A]
Aall = pd.concat(Aall).sort_values(by=["SpesaPer"],ascending=False)
Wall = [WA,W,WT,CWT]
Wall = pd.concat(Wall).sort_values(by=["SpesaPer"],ascending=False)
Tall = [T,TA,WT,CT,CWT]
Tall = pd.concat(Tall).sort_values(by=["SpesaPer"],ascending=False)
Mall =[MC,EM,DdEM]
Mall = pd.concat(Mall).sort_values(by=["SpesaPer"],ascending=False)
Call =[MC,EC,CWT,CT,CW,C]
Call = pd.concat(Call).sort_values(by=["SpesaPer"],ascending=False)
Eall = [EM,EC,DdEM,E,DdDsE,DsE,DdE]
Eall = pd.concat(Eall).sort_values(by=["SpesaPer"],ascending=False)





In [150]:
Dsall =[DsE,DsDc,DdDsE,DdDsDc]
Dsall = pd.concat(Dsall).sort_values(by=["SpesaPer"],ascending=False)
Ddall=[DdEM,DdE,DdDsE,DdDsDc,DdDc]
Ddall = pd.concat(Ddall).sort_values(by=["SpesaPer"],ascending=False)
Dcall = [Dc,DdDsDc,DsDc,DdDc]
Dcall = pd.concat(Dcall).sort_values(by=["SpesaPer"],ascending=False)

In [151]:
MantraDB = pd.ExcelWriter('FinManDBa.xlsx', engine='xlsxwriter')
Attaccanti.to_excel(MantraDB, sheet_name='Attaccanti')
Centrocampisti.to_excel(MantraDB, sheet_name='Centrocampisti')
Difensori.to_excel(MantraDB, sheet_name='Difensori')
Portieri.to_excel(MantraDB, sheet_name='Portieri')
Dcall.to_excel(MantraDB, sheet_name='DcTutti')
Ddall.to_excel(MantraDB, sheet_name='DdTutti')
Dsall.to_excel(MantraDB, sheet_name='DsTutti')
Eall.to_excel(MantraDB, sheet_name='ETutti')
Call.to_excel(MantraDB, sheet_name='CTutti')
Mall.to_excel(MantraDB, sheet_name='MTutti')
Tall.to_excel(MantraDB, sheet_name='TTutti')
Wall.to_excel(MantraDB, sheet_name='WTutti')
Aall.to_excel(MantraDB, sheet_name='ATutti')
Pc.to_excel(MantraDB, sheet_name='PcTutti')
DdDc.to_excel(MantraDB, sheet_name='DdDc')
DdDsDc.to_excel(MantraDB, sheet_name='DdDcDs')
DdDsE.to_excel(MantraDB, sheet_name='DdDsE')
DdE.to_excel(MantraDB, sheet_name='DdE')
DdEM.to_excel(MantraDB, sheet_name='DdEM')
DsDc.to_excel(MantraDB, sheet_name='DsDc')
DsE.to_excel(MantraDB, sheet_name='DsE')
C.to_excel(MantraDB, sheet_name='C')
CT.to_excel(MantraDB, sheet_name='CT')
CW.to_excel(MantraDB, sheet_name='CW')
EC.to_excel(MantraDB, sheet_name='EC')
EM.to_excel(MantraDB, sheet_name='EM')
MC.to_excel(MantraDB, sheet_name='MC')
T.to_excel(MantraDB, sheet_name='T')
W.to_excel(MantraDB, sheet_name='W')
WT.to_excel(MantraDB, sheet_name='WT')
A.to_excel(MantraDB, sheet_name='A')
Dc.to_excel(MantraDB, sheet_name='Dc')
TA.to_excel(MantraDB, sheet_name='TA')
WA.to_excel(MantraDB, sheet_name='WA')
MantraDB.save()
